In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import pandas as pd
import numpy as np

# Setup headless Chrome
options = webdriver.ChromeOptions()
options.add_argument('--headless')
driver = webdriver.Chrome(options=options)

# Go to the page
url = 'https://www.tennisabstract.com/cgi-bin/player-more.cgi?p=106043/Diego-Schwartzman&table=mcp-serve'
driver.get(url)

# Wait for the page to load content (you can adjust this time or use WebDriverWait)
time.sleep(3)

# Get the rendered HTML
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

# Now find the table
table = soup.find('table', id='mcp-serve')
print(table.prettify())

driver.quit()


<table border="0" cellpadding="4" cellspacing="0" class="tablesorter" id="mcp-serve">
 <thead>
  <tr>
   <th align="left">
    <span title="">
     Match
    </span>
   </th>
   <th align="left">
    <span title="">
     Result
    </span>
   </th>
   <th align="right" class="header">
    <span title="Percent of serves that
were unreturned">
     Unret%
    </span>
   </th>
   <th align="right" class="header">
    <span title="Percent of service points
won on either the serve or second shot">
     &lt;=3 W%
    </span>
   </th>
   <th align="right" class="header">
    <span title="Percent of points won when
return was put in play">
     RiP W%
    </span>
   </th>
   <th align="right" class="header">
    <span title="Serve Impact: Advanced stat
estimating how many service points
were won due to the serve">
     SvImpact
    </span>
   </th>
   <th align="right" class="header">
    <span title="Percent of first serves
that were unreturned">
     1st: Unret%
    </span>
   </th>
   <th a

In [2]:
data = []

for row in table.find_all('tr'):
    # Extract the text from each cell in the row
    cells = row.find_all('td')
    if cells:  # Check if the row has any cells
        for cell in cells:
            print(cell.get_text(strip=True))
            data.append((cell.get_text(strip=True)))


2025 Buenos Aires R16
L vsMartinez
9.5%
19.0%
38.2%
16.0%
14.8%
25.9%
34.8%
24.8%
22.2%
38.9%
20.0%
0.0%
9.1%
45.5%
-118
2025 Buenos Aires R32
W vsJarry
26.3%
35.1%
57.5%
31.4%
27.6%
38.2%
58.2%
35.3%
31.3%
47.7%
33.3%
26.5%
32.4%
56.0%
80
2024 US Open R128
L vsMonfils
11.3%
17.4%
47.0%
15.0%
16.2%
24.3%
43.5%
21.9%
54.3%
43.6%
33.3%
2.6%
5.1%
52.6%
-91
2023 Wimbledon R64
L vsSinner
13.6%
17.3%
47.1%
17.7%
15.5%
20.7%
51.0%
21.2%
53.8%
15.6%
16.7%
9.5%
9.5%
36.8%
-22
2023 Barcelona R64
W vsWu
23.8%
31.0%
59.4%
27.9%
25.0%
28.6%
47.6%
31.1%
60.0%
30.8%
-
21.4%
35.7%
81.8%
-79
2023 Monte Carlo R32
L vsSinner
7.7%
7.7%
23.5%
7.7%
14.3%
14.3%
16.7%
14.3%
71.4%
14.3%
50.0%
0.0%
0.0%
40.0%
-118
2023 Indian Wells R64
L vsRuud
15.8%
21.1%
50.0%
19.5%
19.4%
29.0%
60.0%
26.1%
35.7%
41.2%
0.0%
12.5%
12.5%
38.1%
9
2023 Indian Wells R128
W vsCoria
15.6%
20.0%
63.9%
16.7%
17.2%
20.7%
58.3%
19.0%
42.9%
26.7%
0.0%
14.3%
21.4%
75.0%
19
2023 Australian Open R128
W vsKrutykh
27.0%
33.6%
54.2%
31.5%
33.0%

In [3]:
data = [d.replace('\xa0',' ') for d in data]

In [4]:
year_tourney_round = data[::17] # year tournament round in a single string

In [5]:
df_rows = []
for i in range(0, len(data)):
    if i % 17 == 0:
        row = data[i:i+17]
        df_rows.append(row)

In [6]:
cols = ['year tourney round', 'result opponent', 'overall: Unret%', 'overall: <=3 W%', 'overall: RiP W%', 'overall: SvImpact', '1st: Unret%',
        '1st: <=3 W%', '1st: RiP W%', '1st: SvImpact', '1st: D Wide%', '1st: A Wide%', '1st: A BP Wide%', '2nd: Unret%', '2nd: <=3 W%',
        '2nd: RiP W%', '2ndAgg']
df = pd.DataFrame(df_rows, columns=cols)
df['year'] = df['year tourney round'].apply(lambda x: x.split(' ')[0])
df['tourney'] = df['year tourney round'].apply(lambda x: (' ').join(x.split(' ')[1:-1]))
df['round'] = df['year tourney round'].apply(lambda x: x.split(' ')[-1]) 
df['result'] = df['result opponent'].apply(lambda x: x.split(' ')[0])
df['opponent'] = df['result opponent'].apply(lambda x: x.split(' ')[1][2:])
df.drop(columns=['year tourney round', 'result opponent'], inplace=True)
df.head(10)

,overall: Unret%,overall: <=3 W%,overall: RiP W%,overall: SvImpact,1st: Unret%,1st: <=3 W%,1st: RiP W%,1st: SvImpact,1st: D Wide%,1st: A Wide%,1st: A BP Wide%,2nd: Unret%,2nd: <=3 W%,2nd: RiP W%,2ndAgg,year,tourney,round,result,opponent
0,9.5%,19.0%,38.2%,16.0%,14.8%,25.9%,34.8%,24.8%,22.2%,38.9%,20.0%,0.0%,9.1%,45.5%,-118,2025,Buenos Aires,R16,L,Martinez
1,26.3%,35.1%,57.5%,31.4%,27.6%,38.2%,58.2%,35.3%,31.3%,47.7%,33.3%,26.5%,32.4%,56.0%,80,2025,Buenos Aires,R32,W,Jarry
2,11.3%,17.4%,47.0%,15.0%,16.2%,24.3%,43.5%,21.9%,54.3%,43.6%,33.3%,2.6%,5.1%,52.6%,-91,2024,US Open,R128,L,Monfils
3,13.6%,17.3%,47.1%,17.7%,15.5%,20.7%,51.0%,21.2%,53.8%,15.6%,16.7%,9.5%,9.5%,36.8%,-22,2023,Wimbledon,R64,L,Sinner
4,23.8%,31.0%,59.4%,27.9%,25.0%,28.6%,47.6%,31.1%,60.0%,30.8%,-,21.4%,35.7%,81.8%,-79,2023,Barcelona,R64,W,Wu
5,7.7%,7.7%,23.5%,7.7%,14.3%,14.3%,16.7%,14.3%,71.4%,14.3%,50.0%,0.0%,0.0%,40.0%,-118,2023,Monte Carlo,R32,L,Sinner
6,15.8%,21.1%,50.0%,19.5%,19.4%,29.0%,60.0%,26.1%,35.7%,41.2%,0.0%,12.5%,12.5%,38.1%,9,2023,Indian Wells,R64,L,Ruud
7,15.6%,20.0%,63.9%,16.7%,17.2%,20.7%,58.3%,19.0%,42.9%,26.7%,0.0%,14.3%,21.4%,75.0%,19,2023,Indian Wells,R128,W,Coria
8,27.0%,33.6%,54.2%,31.5%,33.0%,41.7%,52.2%,39.7%,67.4%,17.5%,0.0%,15.6%,17.8%,57.9%,39,2023,Australian Open,R128,W,Krutykh
9,12.5%,14.3%,55.8%,15.5%,14.3%,17.1%,53.3%,19.1%,50.0%,17.6%,0.0%,13.3%,13.3%,61.5%,-13,2022,Paris,R64,L,Cressy


In [7]:
neworder = ['year','tourney', 'round', 'result', 'opponent', 'overall: Unret%', 'overall: <=3 W%', 'overall: RiP W%', 'overall: SvImpact', '1st: Unret%', '1st: <=3 W%',
       '1st: RiP W%', '1st: SvImpact', '1st: D Wide%', '1st: A Wide%', '1st: A BP Wide%',
       '2nd: Unret%', '2nd: <=3 W%', '2nd: RiP W%', '2ndAgg']

df = df[neworder]

In [8]:

df.to_csv('../data/schwartzman_serve_chart.csv', index=False) #save to csv in data folder

In [9]:
options = webdriver.ChromeOptions()
options.add_argument('--headless')
driver = webdriver.Chrome(options=options)

# Go to the page
url = 'https://www.tennisabstract.com/cgi-bin/player-more.cgi?p=106043/Diego-Schwartzman&table=serve-speed'
driver.get(url)

# Wait for the page to load content (you can adjust this time or use WebDriverWait)
time.sleep(3)

# Get the rendered HTML
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

# Now find the table
table = soup.find('table', id='serve-speed')
print(table.prettify())

driver.quit()

<table border="0" cellpadding="4" cellspacing="0" class="tablesorter" id="serve-speed">
 <thead>
  <tr>
   <th align="left">
    Match
   </th>
   <th align="left">
    Result
   </th>
   <th align="right" class="header">
    <span title="Average serve speed">
     Avg Speed
    </span>
   </th>
   <th align="right" class="header">
    <span title="Average 1st serve speed">
     1st Avg
    </span>
   </th>
   <th align="right" class="header">
    <span title="Standard deviation of
1st serve speed">
     1st StDev
    </span>
   </th>
   <th align="right" class="header">
    <span t""="" title="Average speed of 1st serves
down the ">
     1st T Avg
    </span>
   </th>
   <th align="right" class="header">
    <span title="Average speed of
1st serves hit wide">
     1st Wide Avg
    </span>
   </th>
   <th align="right" class="header">
    <span title="Fastest second serve">
     Max 1st
    </span>
   </th>
   <th align="right" class="header">
    <span title="Slowest second serve">
  

In [10]:
speed = []

for row in table.find_all('tr'):
    # Extract the text from each cell in the row
    cells = row.find_all('td')
    if cells:  # Check if the row has any cells
        for cell in cells:
            print(cell.get_text(strip=True))
            speed.append((cell.get_text(strip=True)))


2024 US Open R128
L vsMonfils
94.9
100.4
4.8
103.5
97.4
114
92
84.8
4.8
87.6
80.3
95
76
2023 US Open R128
L vsRinderknech
94.2
100.2
4.8
103.8
97.3
114
90
82.6
3.7
79.3

89
75
2023 Wimbledon R64
L vsSinner
97.2
102.7
5.1
107.6
99.5
115
92
82.2
3.6
84.5

88
73
2022 US Open R32
L vsTiafoe
96.6
101.1
5.9
103.6
98.9
115
88
85.3
5.6
81.5
83.0
96
72
2022 US Open R64
W vsPopyrin
94.9
98.8
5.2
102.6
97.1
111
87
85.8
5.0
80.5
82.7
94
72
2022 US Open R128
W vsSock
93.2
98.4
7.1
100.2
96.6
112
73
82.0
3.7
80.0
85.0
89
76
2022 Wimbledon R64
L vsBroady
97.0
102.7
5.0
105.5
101.0
114
91
86.8
4.0
88.0
88.0
95
79
2022 Wimbledon R128
W vsKozlov
95.6
98.8
7.7
100.3
98.8
119
78
85.6
4.7
87.7
88.5
96
75
2021 US Open R16
L vsVan De Zandschulp
98.9
104.3
4.7
106.9
102.4
116
92
86.3
5.0
84.0
89.5
106
78
2021 US Open R32
W vsMolcan
98.0
105.8
5.2
107.3
102.9
114
85
86.1
5.1
91.5

96
74
2021 US Open R64
W vsAnderson
99.7
103.9
3.9
107.3
101.6
114
96
87.0
4.2
87.5

97
80
2021 US Open R128
W vsBerankis
98.8
103.

In [11]:
speed = [s.replace('\xa0',' ') for s in speed]

In [12]:
year_tourney_round = speed[::15]

In [13]:
speed_df_rows = []
for i in range(0, len(speed)):
    if i % 15 == 0:
        row = speed[i:i+15]
        speed_df_rows.append(row)

In [14]:
cols = ['year tourney round', 'result opponent', 'avg speed', '1st: avg', '1st: stdev', '1st: t avg', '1st: wide avg', '1st: max', 
     '1st: min', '2nd: avg', '2nd: stdev', '2nd: t avg', '2nd: wide avg', '2nd: max', '2nd: min']

speed_df = pd.DataFrame(speed_df_rows, columns=cols)
speed_df['year'] = speed_df['year tourney round'].apply(lambda x: x.split(' ')[0])
speed_df['tourney'] = speed_df['year tourney round'].apply(lambda x: (' ').join(x.split(' ')[1:-1]))
speed_df['round'] = speed_df['year tourney round'].apply(lambda x: x.split(' ')[-1])
speed_df['result'] = speed_df['result opponent'].apply(lambda x: x.split(' ')[0])
speed_df['opponent'] = speed_df['result opponent'].apply(lambda x: x.split(' ')[1][2:])
speed_df.drop(columns=['year tourney round', 'result opponent'], inplace=True)
speed_df.head(10)

,avg speed,1st: avg,1st: stdev,1st: t avg,1st: wide avg,1st: max,1st: min,2nd: avg,2nd: stdev,2nd: t avg,2nd: wide avg,2nd: max,2nd: min,year,tourney,round,result,opponent
0,94.9,100.4,4.8,103.5,97.4,114,92,84.8,4.8,87.6,80.3,95,76,2024,US Open,R128,L,Monfils
1,94.2,100.2,4.8,103.8,97.3,114,90,82.6,3.7,79.3,,89,75,2023,US Open,R128,L,Rinderknech
2,97.2,102.7,5.1,107.6,99.5,115,92,82.2,3.6,84.5,,88,73,2023,Wimbledon,R64,L,Sinner
3,96.6,101.1,5.9,103.6,98.9,115,88,85.3,5.6,81.5,83.0,96,72,2022,US Open,R32,L,Tiafoe
4,94.9,98.8,5.2,102.6,97.1,111,87,85.8,5.0,80.5,82.7,94,72,2022,US Open,R64,W,Popyrin
5,93.2,98.4,7.1,100.2,96.6,112,73,82.0,3.7,80.0,85.0,89,76,2022,US Open,R128,W,Sock
6,97.0,102.7,5.0,105.5,101.0,114,91,86.8,4.0,88.0,88.0,95,79,2022,Wimbledon,R64,L,Broady
7,95.6,98.8,7.7,100.3,98.8,119,78,85.6,4.7,87.7,88.5,96,75,2022,Wimbledon,R128,W,Kozlov
8,98.9,104.3,4.7,106.9,102.4,116,92,86.3,5.0,84.0,89.5,106,78,2021,US Open,R16,L,Van
9,98.0,105.8,5.2,107.3,102.9,114,85,86.1,5.1,91.5,,96,74,2021,US Open,R32,W,Molcan


In [15]:
neworder = ['year','tourney', 'round', 'result', 'opponent'] + cols[2:]
speed_df = speed_df[neworder]


In [16]:
for col in speed_df.columns[5:]:
    speed_df[col] = speed_df[col].replace('', np.nan)
speed_df

,year,tourney,round,result,opponent,avg speed,1st: avg,1st: stdev,1st: t avg,1st: wide avg,1st: max,1st: min,2nd: avg,2nd: stdev,2nd: t avg,2nd: wide avg,2nd: max,2nd: min
0,2024,US Open,R128,L,Monfils,94.9,100.4,4.8,103.5,97.4,114,92,84.8,4.8,87.6,80.3,95,76
1,2023,US Open,R128,L,Rinderknech,94.2,100.2,4.8,103.8,97.3,114,90,82.6,3.7,79.3,NaN,89,75
2,2023,Wimbledon,R64,L,Sinner,97.2,102.7,5.1,107.6,99.5,115,92,82.2,3.6,84.5,NaN,88,73
3,2022,US Open,R32,L,Tiafoe,96.6,101.1,5.9,103.6,98.9,115,88,85.3,5.6,81.5,83.0,96,72
4,2022,US Open,R64,W,Popyrin,94.9,98.8,5.2,102.6,97.1,111,87,85.8,5.0,80.5,82.7,94,72
5,2022,US Open,R128,W,Sock,93.2,98.4,7.1,100.2,96.6,112,73,82.0,3.7,80.0,85.0,89,76
6,2022,Wimbledon,R64,L,Broady,97.0,102.7,5.0,105.5,101.0,114,91,86.8,4.0,88.0,88.0,95,79
7,2022,Wimbledon,R128,W,Kozlov,95.6,98.8,7.7,100.3,98.8,119,78,85.6,4.7,87.7,88.5,96,75
8,2021,US Open,R16,L,Van,98.9,104.3,4.7,106.9,102.4,116,92,86.3,5.0,84.0,89.5,106,78
9,2021,US Open,R32,W,Molcan,98.0,105.8,5.2,107.3,102.9,114,85,86.1,5.1,91.5,NaN,96,74


In [17]:
for col in speed_df.columns[5:]:
    speed_df[col] = speed_df[col].astype(float)

In [18]:
speed_df.to_csv('../data/schwartzman_serve_speed.csv', index=False) #change the name of the file to save it as you want